# Recomendador de libros en Goodreads

En este proyecto elaboré un recomendador de libros con base el perfil de un usuario en Goodreads. En esencia, el programa le sugiere a un usuario cuál es próximo libro que debe leer. 

Para recopilar la información que alimentará al recomendador, utilicé la API de Goodreads. 

#### Algunos aspectos que me gustaría incluir después:
- A

#### Mejoras pendientes a mi código:
- Manejar errores, con try/except, en los GET requests.
- Optimizar mi función principal para que se vea más limpia con loops. No super cómo hacerlo, considerando que debo cambiar de página y cambia el código html. Una vez que logre hacer esto, podré agregarle, también, un parametro para elegir las etiquetas que queremos seleccionar.


## I. Obtener información de un perfil

In [2]:
# Importar librerías
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import numpy as np

In [3]:
# Claves para acceder al API
CONSUMER_KEY = 'PnnwwD2CW9RtBv6YqsjYw'
CONSUMER_SECRET = 'S1jfjAZpNn1dH6iRBWgDINoFp8G2ucwJa1C2OHg70xw'
url = "https://www.goodreads.com/"
id_francisco_galan = '40732498'

### 1. Libros leídos de un usuario

Las siguientes funciones (en particular `extract_info`) permiten extraer toda la información del estante de libros leídos de un usuario. La información se arroja como un diccionario.

In [3]:
def shelf_info(user_id, shelf, page):
  
    """Arroja un BeautifulSoup object de una página del librero de un usuario"""
    
    time.sleep(1.1) # Para cumplir con los términos y condiciones de Goodreads
    info = requests.get(f'{url}/review/list?v=2&id={user_id}&shelf={shelf}&sort=title&page={page}&per_page=200&key={CONSUMER_KEY}')
    print('Status code: ', info.status_code)
    info_content = info.content
    soup = BeautifulSoup(info_content, 'lxml')
    return soup

def paginas_por_estante(libros_en_estante):
    """Nos dice cuántas páginas se requieren para mostrar todos los libros del estante"""
    
    if libros_en_estante <= 200:
        return 1
    
    elif libros_en_estante <= 400:
        return 2
    
    elif libros_en_estante <= 600:
        return 3
    
    elif libros_en_estante <= 800:
        return 4
    
    elif libros_en_estante <= 1000:
        return 5
    
    elif libros_en_estante <= 1200:
        return 6

def extract_info(user_id, shelf, libros_en_estante):
    
    diccionario = {}
    
    # Definir variables de mis tags para ir agregando información mientras avanzo por varias páginas.
    isbn = []
    title = []
    num_pages = []
    publisher = []
    publication_year = []
    name = []
    rating = []
    average_rating = []
    ratings_count = []
    links = []
    
    #Determinar cuántas páginas consideraremos dentro del estante.
    paginas_to_scrap = paginas_por_estante(libros_en_estante)
    
    # Extraer información de cada una de las páginas
    for x in range(1, paginas_to_scrap+1):
        
        # Obtener código html en formato Soup
        page = shelf_info(user_id, shelf=shelf, page=x)
        
        # Obtener información de tags que no se repiten
        
        isbn_info = page.find_all(f'isbn')
        isbn_info = [elem.get_text() for elem in isbn_info]
        isbn.append(isbn_info)
        
        title_info = page.find_all(f'title')
        title_info = [elem.get_text() for elem in title_info]
        title.append(title_info)
        
        publisher_info = page.find_all(f'publisher')
        publisher_info = [elem.get_text() for elem in publisher_info]
        publisher.append(publisher_info)
        
        num_pages_info = page.find_all(f'num_pages')
        num_pages_info = [elem.get_text() for elem in num_pages_info]
        num_pages.append(num_pages_info)
        
        publication_year_info = page.find_all(f'publication_year')
        publication_year_info = [elem.get_text() for elem in publication_year_info]
        publication_year.append(publication_year_info)
        
        name_info = page.find_all(f'name')
        name_info = [elem.get_text() for elem in name_info]
        name.append(name_info)
        
        rating_info = page.find_all(f'rating')
        rating_info = [elem.get_text() for elem in rating_info]
        rating.append(rating_info)
        
        # Obtener información de tags que se repiten
        review_blocks = page.find_all('review')
        link_pattern = re.compile(r'www.goodreads.com.*')
        
        for review in review_blocks: 
            average_rating_info = review.find(f'average_rating').get_text()
            average_rating.append(average_rating_info)

            ratings_count_info = review.find(f'ratings_count').get_text()
            ratings_count.append(ratings_count_info)
            
            # Obtener links        
            if link_pattern.search(review.get_text()):
                link = re.findall(link_pattern, review.get_text()) 
                links.append(link)
    
            else: 
                print('Missing: ', review.title)
    
    # Aplanar variables con listas dentro de una lista
    isbn = [elem for listt in isbn for elem in listt]
    title = [elem for listt in title for elem in listt]
    num_pages = [elem for listt in num_pages for elem in listt]
    publisher = [elem for listt in publisher for elem in listt]
    publication_year = [elem for listt in publication_year for elem in listt]
    name = [elem for listt in name for elem in listt]
    rating = [elem for listt in rating for elem in listt]
    links = [elem[0] for elem in links]
    
    # Pasar todo a un diccionario
    diccionario[f'user_id'] = [user_id for x in range(0, len(isbn))]
    diccionario[f'shelf'] = [shelf for x in range(0, len(isbn))]
    
    diccionario[f'isbn'] = isbn
    diccionario[f'title'] = title
    diccionario[f'author'] = name
    diccionario[f'num_pages'] = num_pages
    diccionario[f'publication_year'] = publication_year
    diccionario[f'publisher'] = publisher
    diccionario[f'my_rating'] = rating
    
    diccionario[f'average_rating'] = average_rating
    diccionario[f'ratings_count'] = ratings_count
    diccionario[f'links'] = links
        
    return diccionario


### 2. Lista de amigos del usuario

In [4]:
from rauth.service import OAuth1Service, OAuth1Session

ModuleNotFoundError: No module named 'rauth'

In [12]:
time.sleep(1.1)
info = requests.get(f'{url}friend/user.xml&id={id_francisco_galan}&page=1-10&sort=first_name&key={CONSUMER_KEY}')
print('Status code: ', info.status_code)

Status code:  200


In [13]:
info_content = info.content
soup = BeautifulSoup(info_content, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Sign Up
  </title>
  <meta content="telephone=no" name="format-detection"/>
  <link href="https://www.goodreads.com/user/sign_up" rel="canonical"/>
  <!-- OPEN GRAPH-->
  <meta content="Goodreads" property="og:title"/>
  <meta content="website" property="og:type"/>
  <meta content="Goodreads" property="og:site_name"/>
  <meta content="https://s.gr-assets.com/assets/icons/goodreads_icon_100x100-4a7d81b31d932cfc0be621ee15a14e70.png" property="og:image"/>
  <meta content="2415071772" property="fb:app_id"/>
  <!-- Helps the FB app link to our official iOS/Android app -->
  <meta content="com.goodreads.https://user/sign_up?launchedbyapplink=true" property="al:ios:url"/>
  <meta content="355833469" property="al:ios:app_store_id"/>
  <meta content="Goodreads'" property="al:ios:app_name"/>
  <script type="text/javascript">
   if (window.Mobvious === undefined) {
          window.Mobvious = {};
        }
        window.Mobvious.device_type = 'desktop'

#### a) Estante de libros leídos

In [ ]:
leido_francisco_galan = extract_info(id_francisco_galan, 'read', 276)

#### b) Estante de libros por leer

In [ ]:
por_leer_francisco_galan = extract_info(id_francisco_galan, 'to-read', 104)

## II. Obtener bases de datos en pandas

Ahora ya tenemos una función que almacena toda la información del estante de un usuario en un diccionario. Procedamos ahora a pasar la información a una base de datos en pandas.

### 2.1 Generar DataFrames

Antes de pasar la información a formato DataFrame en pandas, chequemos que todas las variables tengan la misma extensión:

In [ ]:
def check_variables_length(dictionary):
    for x, y in dictionary.items():
        print(f'{x}: ', len(y))

In [ ]:
check_variables_length(leido_francisco_galan)

Ahora sí, pasemos los datos a un DataFrame

In [ ]:
pd_leido = pd.DataFrame(leido_francisco_galan)
pd_leido

In [ ]:
check_variables_length(por_leer_francisco_galan)

In [ ]:
pd_por_leer = pd.DataFrame(por_leer_francisco_galan)
pd_por_leer

### 2.2 Unir DataFrames

Antes de analizar la información, comencemos por unir los dos DataFrames para tenerlos en un mismo lugar:

In [ ]:
pd_total = pd.concat([pd_leido, pd_por_leer], axis=0)
pd_total

In [ ]:
# Resetear índices
pd_total = pd_total.reset_index(drop=True)

### 2.3 Explorar información

In [ ]:
pd_total.info()

Podemos observar que nuestras 11 columnas están como tipo `object`, lo cual no es adecuado. 

In [ ]:
pd_total.isnull().sum()

Vemos también que no hay valores nulos. No obstante, esto es engañoso porque sí parece haber celdas vacías, como se muestra a continuación en las columnas`isbn` o `num_pages`:

In [ ]:
pd_total.head(5)

In [ ]:
pd_total.tail(5)

In [ ]:
print(pd_total['isbn'][1])
print(type(pd_total['isbn'][1]))

Parece que estas celdas son simplemente strings vacíos. Transformémoslas a valores `NaN` para analizar mejor la información:

In [ ]:
pd_total = pd_total.replace(r'^\s*$', np.NaN, regex=True)
pd_total.isnull().sum()

Vemos que hay cuatro columnas en las que faltan valores, aunque será difícil llenarlos, puesto que no nos podemos basar en la demás información de la base de datos.

In [ ]:
pd_total.describe()

Verifiquemos que no hay valores outliers en las columnas:

### 2.4 Cambiar tipos de variables

Cambiemos aquellas columnas que son puramente numéricas:

In [ ]:
pd_total['num_pages'] = pd_total['num_pages'].astype('float')
pd_total['my_rating'] = pd_total['my_rating'].astype('float')
pd_total['average_rating'] = pd_total['average_rating'].astype('float')
pd_total['ratings_count'] = pd_total['average_rating'].astype('float')

Ahora cambiemos la columna con la fecha de publicación. Primero, veamos en qué formato están los valores:

In [ ]:
pd_total['publication_year'].value_counts()

In [ ]:
pd_total['publication_year'] = pd.to_datetime(pd_total['publication_year'], format='%Y')

In [ ]:
pd_total.info()

### 2.5 Limpiar columna de `my_rating`

En esta columna, a los libros que no les di calificación les asigna un 0. Conviene, mejor, pasar estos valores a NaN para que no alteren otros cálculos:

In [ ]:
pd_total['my_rating'] = pd_total['my_rating'].replace(0, np.NaN)

In [ ]:
pd_total['my_rating'].isnull().sum()

### 2.6 Exportar como Excel

In [ ]:
pd_total.to_excel(r'Output/Francisco_galan.xlsx', index=False)

In [ ]:
pd_total.to_csv(f'Output/francisco_galan.csv', index=False, )

## III. Extra

A continuación, algunas conclusiones sobre mis hábitos de lectura.



Extensión promedio de los libros que he leído.

In [ ]:
int(pd_total['num_pages'].mean())

Los cinco autores que más he leído:

In [ ]:
pd_total['author'].value_counts().head(5)

Calificación que doy en promedio:

In [ ]:
pd_total['my_rating'].mean()